In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Data uploading

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.rename(columns={'Time, seconds':'Time','Series values, dB':'dB'},inplace=True)

In [ ]:
plt.plot(df['Time'], df['dB'])

In [ ]:
sns.displot(df['dB'])

In [ ]:
df['dB'].mean()

In [ ]:
df['dB'].median()

In [ ]:
df['dB'].std()

In [ ]:
df['dB'].skew()

In [ ]:
df['dB'].kurtosis()

# Checking Normality

## Q-Q plot

In [ ]:
import scipy as sp

f,ax = plt.subplots()
_,(_,_,r)= sp.stats.probplot(df['dB'],plot=ax)
ax.set_title('Q-Q Plot')
plt.show()

## Statistical tests, Shapiro-Wilk and Ktest

In [ ]:
from scipy.stats import kstest, norm
ks_statistic, p_value = kstest(df['dB'], 'norm')
print(ks_statistic, p_value)

In [ ]:
from scipy.stats import shapiro
shapiro(df['dB'])

## Checking with Z score value

In [ ]:
mean = df['dB'].mean()
std = df['dB'].std()

upper_bound= mean + 3*std
lower_bound= mean - 3*std

plt.plot('Time','dB',data=df)
plt.axhline(lower_bound,color='r')
plt.axhline(upper_bound,color='r')

In [ ]:
low = np.percentile(df['dB'],0.3)
up = np.percentile(df['dB'],99.7)
plt.plot('Time','dB',data=df)
plt.axhline(low,color='r')
plt.axhline(up,color='r')

In normal distribution, "mean + 3*std" should be equal to the 99.7th percentile value, but graphs does not match. It means data is not normal distributed.

# Anomaly detection with statistical threshold

In [ ]:
low = np.percentile(df['dB'],0.2)
upper = np.percentile(df['dB'],99.7)

plt.plot('Time','dB',data=df)
plt.axhline(low,color='r')
plt.axhline(upper,color='r')

In [ ]:
q1, q3 = np.percentile(df['dB'],[25,75])
IQR = q3-q1
low = q1 - IQR*1.5
up = q3 + 1.5*IQR

In [ ]:
plt.plot('Time','dB',data=df)
plt.axhline(low,color='r')
plt.axhline(up,color='r')

IQR method gives lower threshold for anomalies. Thus, i will apply moving average to denoise data.

In [ ]:
data = df.copy()
ma = 10
data['dB'] = data['dB'].rolling(ma).mean()
data = data.loc[ma:,:]
q1, q3 = np.percentile(data['dB'],[25,75])
IQR = q3-q1
low = q1 - IQR*1.5
up = q3 + 1.5*IQR

In [ ]:
plt.plot('Time','dB',data=data)
plt.axhline(low,color='r')
plt.axhline(up,color='r')

# Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [ ]:
# Extract the feature matrix from the DataFrame
standard_scaler = StandardScaler()
np_scaled = standard_scaler.fit_transform(df[['dB']])
X = pd.DataFrame(np_scaled)
# Initialize the Isolation Forest model with some hyperparameters
model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01, random_state=42)

# Fit the model to the data
model.fit(X)

# Use the model to predict which data points are anomalies
df['anomaly25'] = pd.Series(model.predict(X))
df['anomaly25'] = df['anomaly25'].map( {1: 0, -1: 1} )
print(df['anomaly25'].value_counts())

In [ ]:
fig, ax = plt.subplots()

a = df.loc[df['anomaly25'] == 1, ['Time', 'dB']] #anomaly

ax.plot(df['Time'], df['dB'], color='blue')
ax.scatter(a['Time'],a['dB'], color='red')
plt.show()

In [ ]:
data = df.copy()
ma = 10
data['dB'] = data['dB'].rolling(ma).mean()
data = data.loc[ma:,:]

In [ ]:
# min_max_scaler = MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(data[['dB']])
# X = pd.DataFrame(np_scaled)
# Initialize the Isolation Forest model with some hyperparameters
model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.011, random_state=42)

# Fit the model to the data
model.fit(data[['dB']])

# Use the model to predict which data points are anomalies
data['anomaly25'] = pd.Series(model.predict(data[['dB']]))
data['anomaly25'] = data['anomaly25'].map( {1: 0, -1: 1} )
print(data['anomaly25'].value_counts())

In [ ]:
fig, ax = plt.subplots()

a = data.loc[data['anomaly25'] == 1, ['Time', 'dB']] #anomaly

ax.plot(data['Time'], data['dB'], color='blue')
ax.scatter(a['Time'],a['dB'], color='red')
plt.show()

In [ ]:
fig, ax = plt.subplots()

a = data.loc[data['anomaly25'] == 1, ['Time', 'dB']] #anomaly

ax.plot(data['Time'], data['dB'], color='blue')
ax.scatter(a['Time'],a['dB'], color='red')
plt.show()

# Autoencoder

In [ ]:
df1 = df.copy()

In [ ]:
df1 = df1[df1['Time']<10]

In [ ]:
plt.plot(df1['Time'],df1['dB'])

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Input, Dropout
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from matplotlib import pyplot as plt
from keras.models import Model

#Change train data for 5 seconds
train, test = df.loc[df['Time'] <= 10], df.loc[df['Time'] > 10]

scaler = StandardScaler()
scaler = scaler.fit(train[['dB']])

train['dB'] = scaler.transform(train[['dB']])
test['dB'] = scaler.transform(test[['dB']])


seq_size = 10  # Number of time steps to look back 
#Larger sequences (look further back) may improve forecasting.


def to_sequences(x, y, seq_size=1):
    x_values = []
    y_values = []

    for i in range(len(x)-seq_size):
        #print(i)
        x_values.append(x.iloc[i:(i+seq_size)].values)
        y_values.append(y.iloc[i+seq_size])
        
    return np.array(x_values), np.array(y_values)

trainX, trainY = to_sequences(train[['dB']], train['dB'], seq_size)
testX, testY = to_sequences(test[['dB']], test['dB'], seq_size)


# define Autoencoder model
#Input shape would be seq_size, 1 - 1 beacuse we have 1 feature. 
seq_size = trainX.shape[1]

model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(trainX.shape[1]))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(trainX.shape[2])))

model.compile(optimizer='adam', loss='mse')
model.summary()

#Try another model
# model = Sequential()
# model.add(LSTM(128, input_shape=(trainX.shape[1], trainX.shape[2])))
# model.add(Dropout(rate=0.2))

# model.add(RepeatVector(trainX.shape[1]))

# model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(rate=0.2))
# model.add(TimeDistributed(Dense(trainX.shape[2])))
# model.compile(optimizer='adam', loss='mae')
# model.summary()

# fit model
history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

model.evaluate(testX, testY)


In [ ]:
#Anomaly is where reconstruction error is large.
#We can define this value beyond which we call anomaly.
#Let us look at MAE in training prediction

trainPredict = model.predict(trainX)
trainMAE = np.mean(np.abs(trainPredict - trainX), axis=1)
plt.hist(trainMAE, bins=30)

In [ ]:
max_trainMAE = 3
testPredict = model.predict(testX)
testMAE = np.mean(np.abs(testPredict - testX), axis=1)
plt.hist(testMAE, bins=30)

In [ ]:
#Capture all details in a DataFrame for easy plotting
anomaly_df = pd.DataFrame(test[seq_size:])
anomaly_df['testMAE'] = testMAE
anomaly_df['max_trainMAE'] = max_trainMAE
anomaly_df['anomaly'] = anomaly_df['testMAE'] > anomaly_df['max_trainMAE']
anomaly_df['dB'] = test[seq_size:]['dB']

#Plot testMAE vs max_trainMAE
sns.lineplot(x=anomaly_df['Time'], y=anomaly_df['testMAE'])
sns.lineplot(x=anomaly_df['Time'], y=anomaly_df['max_trainMAE'])

anomalies = anomaly_df.loc[anomaly_df['anomaly'] == True]

In [ ]:
#Plot anomalies
plt.plot(anomaly_df['Time'], scaler.inverse_transform(anomaly_df[['dB']]))
plt.scatter(x=anomalies['Time'], y=scaler.inverse_transform(anomalies[['dB']]), color='r')

In [ ]:
anomalies.head()